In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

np.random.seed(2)
onehot_encoder = OneHotEncoder(sparse=False)

In [23]:
data = pd.read_csv('Data/iris.csv')

labelencoder = LabelEncoder()
data["species"] = labelencoder.fit_transform(data["species"])
X = data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y = data['species']

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2)
X_train = X_train.values.T
X_test = X_test.values.T
Y_train = Y_train.values.T
Y_test = Y_test.values.T

In [25]:
print("|SHAPES| |X_train:", np.shape(X_train),"|X_test:", np.shape(X_test), "|Y_train:", np.shape(Y_train),  
      "|Y_test:", np.shape(Y_test))

|SHAPES| |X_train: (4, 120) |X_test: (4, 30) |Y_train: (120,) |Y_test: (30,)


In [26]:
onehot = onehot_encoder.fit_transform(Y_train.reshape(len(Y_train), 1))
Y_train = onehot.T
print("Y_train shape: ", np.shape(Y_train))

Y_train shape:  (3, 120)


In [27]:
def initialize_parameters(n_x, n_h, n_y):
    
    W1 = np.random.randn(n_h, n_x) * 0.01 #(6,4)
    b1 = np.random.randn(n_h, 1) * 0.01 #(6,1)
    W2 = np.random.randn(n_y, n_h) * 0.01 #(3,6)
    b2 = np.random.randn(n_y, 1) * 0.01 #(3,1)

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [28]:
def forward_propagation(X, parameters):

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)  
    Z2 = np.dot(W2, A1) + b2
    
    #stabilizzazione (trovato su internet (devo approfondire))
    #serve per evitare: nan error due to floating point limitation in NumPy 
    X = Z2 - np.max(Z2, axis=0, keepdims=True)
    exps = np.exp(X)
    A2 = exps/np.sum(exps,axis=0,keepdims=True)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [29]:
def cost(A2, Y, parameters):
    
    m = Y.shape[1]
    
    #costo cross-entropy
    logprobs = np.sum(Y * np.log(A2 + 1e-8), axis=0)
    cost = - np.sum(logprobs) / m
    
    return cost

In [30]:
def sigmoid_derivative(Z):
    s = 1 / (1 + np.exp(-Z))
    return s * (1 - s)

In [31]:
def backward_propagation(parameters, cache, X, Y):
    
    m = Y.shape[1]
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    A1 = cache['A1']
    A2 = cache['A2']
    Z1 = cache['Z1']
    
    dZ2= A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * sigmoid_derivative(Z1)
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dW1": dW1,
                 "db1": db1,
                 "dW2": dW2,
                 "db2": db2}
        
    return gradients

In [32]:
def update_parameters(parameters, gradients, learning_rate=1.2):

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    dW1 = gradients['dW1']
    db1 = gradients['db1']
    dW2 = gradients['dW2']
    db2 = gradients['db2']
    
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [33]:
def train_model(X, Y, n_h, num_iterations=10000, print_cost=False):
    
    n_x = X.shape[0]
    n_y = Y.shape[0]
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    #gradient descent
    for i in range(0, num_iterations):
        A2, cache = forward_propagation(X, parameters)
        computed_cost = cost(A2, Y, parameters)
        gradients = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, gradients)
        
        if (print_cost and i % 500 == 0) or (print_cost and i == num_iterations-1):
            print ("Costo dopo l'iterazione %i: %f" % (i, computed_cost))
            
    return parameters,n_h

In [34]:
parameters,_ = train_model(X_train,Y_train , n_h = 6, num_iterations=10000, print_cost=True)

Costo dopo l'iterazione 0: 1.098233
Costo dopo l'iterazione 500: 0.892678
Costo dopo l'iterazione 1000: 0.180617
Costo dopo l'iterazione 1500: 0.171843
Costo dopo l'iterazione 2000: 0.150761
Costo dopo l'iterazione 2500: 0.112556
Costo dopo l'iterazione 3000: 0.096723
Costo dopo l'iterazione 3500: 0.088373
Costo dopo l'iterazione 4000: 0.084460
Costo dopo l'iterazione 4500: 0.081879
Costo dopo l'iterazione 5000: 0.079987
Costo dopo l'iterazione 5500: 0.078530
Costo dopo l'iterazione 6000: 0.084787
Costo dopo l'iterazione 6500: 0.068499
Costo dopo l'iterazione 7000: 0.067273
Costo dopo l'iterazione 7500: 0.067653
Costo dopo l'iterazione 8000: 0.073197
Costo dopo l'iterazione 8500: 0.084153
Costo dopo l'iterazione 9000: 0.064833
Costo dopo l'iterazione 9500: 0.065721
Costo dopo l'iterazione 9999: 0.083896


In [35]:
onehot_encoder = OneHotEncoder(sparse=False)
onehot = onehot_encoder.fit_transform(Y_test.reshape(len(Y_test), 1))
Y_test = onehot.T
print("Y_test shape: ", np.shape(Y_test))

Y_test shape:  (3, 30)


In [36]:
Y_hat,_ = forward_propagation(X_test, parameters)

In [37]:
y_hat = np.argmax(Y_hat, axis=0)
y = np.argmax(Y_test, axis=0)

In [38]:
accuracy = ( y_hat == y).mean() * 100

In [39]:
print(accuracy,"%")

96.66666666666667 %
